In [1]:
import pandas as pd
import os
import time
import csv
import re

`Configuration Dataset`

In [ ]:
raw_data_dir = 'dataset'
cleaned_data_dir = 'datasets_cleaned'
file_to_clean_name = 'etd_ugm.csv'

input_filepath = os.path.join(raw_data_dir, file_to_clean_name)
output_filepath = os.path.join(cleaned_data_dir, file_to_clean_name)

print("Konfigurasi Selesai:")
print(f"  File Input Target : {input_filepath}")
print(f"  File Output       : {output_filepath}")


In [ ]:
def clean_and_quote_csv(input_filepath, output_filepath):
    os.makedirs(os.path.dirname(output_filepath), exist_ok=True)
    
    df = pd.read_csv(input_filepath)
    
    df = df.replace("–", "-", regex=True)
    
    if 'judul' in df.columns:
        df['judul'] = df['judul'].apply(
            lambda x: f'"{x}"' if not (str(x).startswith('"') and str(x).endswith('"')) else x
        )
    
    df.to_csv(output_filepath, index=False, encoding='utf-8')
    
    print("\nProses pembersihan selesai!")
    print(f"File bersih disimpan di: {output_filepath}")

input_filepath = os.path.join(raw_data_dir, file_to_clean_name)
output_filepath = os.path.join(cleaned_data_dir, file_to_clean_name)

print("Konfigurasi Selesai:")
print(f"  File Input Target : {input_filepath}")
print(f"  File Output       : {output_filepath}")

clean_and_quote_csv(input_filepath, output_filepath)

In [ ]:
def clean_and_quote_csv(input_filepath, output_filepath):
    os.makedirs(os.path.dirname(output_filepath), exist_ok=True)
    
    df = pd.read_csv(input_filepath)
    
    df = df.replace("–", "-", regex=True)
    
    if 'judul' in df.columns:
        df['judul'] = df['judul'].apply(
            lambda x: f'"{x}"' if not (str(x).startswith('"') and str(x).endswith('"')) else x
        )
    
    df.to_csv(output_filepath, index=False, encoding='utf-8')
    
    print("\nProses pembersihan selesai!")
    print(f"File bersih disimpan di: {output_filepath}")

# Konfigurasi path
raw_data_dir = 'dataset'
cleaned_data_dir = 'datasets_cleaned'
file_to_clean_name = 'etd_usk.csv'

input_filepath = os.path.join(raw_data_dir, file_to_clean_name)
output_filepath = os.path.join(cleaned_data_dir, file_to_clean_name)

print("Konfigurasi Selesai:")
print(f"  File Input Target : {input_filepath}")
print(f"  File Output       : {output_filepath}")

# Jalankan proses pembersihan
clean_and_quote_csv(input_filepath, output_filepath)

`Formating untuk dataset khusus etd_ugm`

In [ ]:
def fix_all_formats(input_filepath, output_filepath):
    os.makedirs(os.path.dirname(output_filepath), exist_ok=True)
    
    data = []
    
    with open(input_filepath, 'r', encoding='utf-8', errors='ignore') as f:
        content = f.read()
    
    entries = re.split(r'\n(?=[^",\n]+,")', content)
    
    if len(entries) <= 1:
        entries = re.split(r'\n"(?=[^"]+","[^"])', content)
    
    for entry in entries:
        entry = entry.strip()
        if not entry or 'judul' in entry.lower():
            continue
        
        judul, konten = parse_entry(entry)
        
        if judul and konten:
            judul = clean_text(judul)
            konten = clean_text(konten)
            
            data.append({'judul': judul, 'konten': konten})
    
    df = pd.DataFrame(data)
    
    df.to_csv(output_filepath, index=False, encoding='utf-8')
    
    print(f"✅ Formatting selesai!")
    print(f"   Data diproses: {len(data)} baris")
    print(f"   File disimpan: {output_filepath}")
    
    if len(df) > 0:
        print(f"\n📋 Sample hasil:")
        print("=" * 80)
        for i in range(min(3, len(df))):
            print(f"Data {i+1}:")
            print(f"  Judul: {df['judul'].iloc[i][:80]}...")
            print(f"  Konten: {df['konten'].iloc[i][:80]}...")
            print()

def parse_entry(entry):
    entry = entry.strip()
    
    if re.match(r'^[^"]+,"[^"]+', entry):
        first_quote = entry.find('"')
        judul = entry[:first_quote-1].strip() if first_quote > 0 else ""
        konten = entry[first_quote:].strip()
        
        if konten.startswith('"') and konten.endswith('"'):
            konten = konten[1:-1]
        elif konten.startswith('"'):
            konten = konten[1:]
        elif konten.endswith('"'):
            konten = konten[:-1]
            
        return judul, konten

    elif re.match(r'^"[^"]+","[^"]+', entry):
        parts = entry.split('","', 1)
        if len(parts) == 2:
            judul = parts[0][1:].strip() if parts[0].startswith('"') else parts[0].strip()
            konten = parts[1][:-1].strip() if parts[1].endswith('"') else parts[1].strip()
            return judul, konten
    
    elif entry.startswith('"""') and entry.endswith('"""'):
        entry = entry[3:-3] 
        return parse_entry(entry)  
    
   
    elif ',"""' in entry and entry.endswith('"""'):
        parts = entry.split(',"""', 1)
        if len(parts) == 2:
            judul = parts[0].strip()
            konten = parts[1][:-3].strip() 
            return judul, konten
    
   
    elif ',' in entry:
        first_comma = entry.find(',')
        judul = entry[:first_comma].strip()
        konten = entry[first_comma+1:].strip()
        
       
        judul = judul.strip('"')
        konten = konten.strip('"')
        
        return judul, konten
    
    return None, None

def clean_text(text):
    if not text:
        return ""
    
   
    text = re.sub(r'"+', '"', text)
    
   
    text = text.strip()
    while text.startswith('"') and text.endswith('"'):
        text = text[1:-1].strip()
    
   
    text = text.replace("–", "-")
    text = text.replace("”", '"')
    text = text.replace("“", '"')
    
   
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

def process_complex_file(input_filepath, output_filepath):
    os.makedirs(os.path.dirname(output_filepath), exist_ok=True)
    
    data = []
    current_entry = []
    
    with open(input_filepath, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            line = line.strip()
            
            if not line or 'judul' in line.lower():
                continue
            
           
            if re.match(r'^[^",]+,"[^"]+', line) and current_entry:
                process_current_entry(current_entry, data)
                current_entry = []
            
            current_entry.append(line)
        
       
        if current_entry:
            process_current_entry(current_entry, data)
    
    
    df = pd.DataFrame(data)
    
   
    df.to_csv(output_filepath, index=False, encoding='utf-8')
    
    print(f"✅ Complex formatting selesai!")
    print(f"   Data diproses: {len(data)} baris")
    print(f"   File disimpan: {output_filepath}")

def process_current_entry(entry_lines, data):
  
    full_entry = ' '.join(entry_lines)
    judul, konten = parse_entry(full_entry)
    
    if judul and konten:
        judul = clean_text(judul)
        konten = clean_text(konten)
        data.append({'judul': judul, 'konten': konten})


input_filepath = os.path.join('dataset', 'etd_ugm.csv')
output_filepath = os.path.join('datasets_cleaned', 'etd_ugm.csv')

print("Memulai proses formatting...")
print(f"  Input : {input_filepath}")
print(f"  Output: {output_filepath}")
try:
    fix_all_formats(input_filepath, output_filepath)
except Exception as e:
    print(f"❌ Method 1 failed: {e}")
    print("Trying complex method...")
    try:
        process_complex_file(input_filepath, output_filepath)
    except Exception as e2:
        print(f"❌ All methods failed: {e2}")

## Cleanup Dataset

`Mojok-Dataset` - `Fix`

In [ ]:
def clean_excessive_quotes(text):
  
    if not text:
        return text
    
    text = text.strip()
    
    if text.count('"') >= 4:
        print(f"DEBUG: Banyak kutip ditemukan: {text[:100]}...")
    
  
    while text.startswith('"""'):
        text = text[3:]
    while text.startswith('""'):
        text = text[2:]
    if text.startswith('"'):
        text = text[1:]
    
    while text.endswith('"""'):
        text = text[:-3]
    while text.endswith('""'):
        text = text[:-2]
    if text.endswith('"'):
        text = text[:-1]
    
    return text.strip()

def remove_source_prefix(text):
   
    if not text:
        return text
    
  
    patterns = [
        r'^"\s*KOMPAS\.com\s*[–\-]\s*',
        r'^KOMPAS\.com\s*[–\-]\s*',
        r'^"\s*KOMPAS\.com\s*',
        r'^KOMPAS\.com\s*',
        r'^\(\s*KOMPAS\.com\s*\)\s*',
        r'^\[\s*KOMPAS\.com\s*\]\s*'
    ]
    
    cleaned_text = text.strip()
    
    for pattern in patterns:
        if re.match(pattern, cleaned_text, re.IGNORECASE):
            cleaned_text = re.sub(pattern, '', cleaned_text, flags=re.IGNORECASE)
            break
    
   
    cleaned_text = cleaned_text.replace('""', '"')
    
    return cleaned_text.strip()

def parse_csv_line_robust(line):
    line = line.strip()
    
    
    try:
        reader = csv.reader([line], quotechar='"', delimiter=',', quoting=csv.QUOTE_ALL, skipinitialspace=True)
        row = next(reader)
        if len(row) == 2:
            return [clean_excessive_quotes(row[0]), clean_excessive_quotes(row[1])]
    except Exception as e:
        pass
    
   
    match = re.match(r'^"([^"]*)"\s*,\s*"([^"]*)"$', line)
    if match:
        judul = clean_excessive_quotes(match.group(1))
        konten = clean_excessive_quotes(match.group(2))
        return [judul, konten]
    
   
    if line.count('"') >= 4:
       
        parts = re.split(r',\s*"', line, 1)
        if len(parts) == 2:
            judul = clean_excessive_quotes(parts[0])
            konten = '"' + parts[1]  
            konten = clean_excessive_quotes(konten)
            return [judul, konten]
    
    
    if ',' in line:
        first_comma = line.find(',')
        judul = line[:first_comma].strip()
        konten = line[first_comma+1:].strip()
        
        judul = clean_excessive_quotes(judul)
        konten = clean_excessive_quotes(konten)
        
        return [judul, konten]
    
    return None

def ensure_proper_quotes(judul, konten):
    
    judul = clean_excessive_quotes(judul)
    konten = clean_excessive_quotes(konten)
    
    konten = remove_source_prefix(konten)
    
    judul = judul.strip('"')
    konten = konten.strip('"')
    
    return judul, konten

def clean_and_split_csv(input_path, output_path):
    print(f"-> Memulai pembersihan untuk: {os.path.basename(input_path)}")

    success_count = 0
    fail_count = 0
    failed_lines = []

    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    with open(input_path, 'r', encoding='utf-8', errors='ignore') as infile, \
         open(output_path, 'w', encoding='utf-8', newline='') as outfile:

        writer = csv.writer(outfile, quoting=csv.QUOTE_ALL)  
        writer.writerow(['judul', 'konten'])

        for line_num, line in enumerate(infile, 1):
            line = line.strip()
            
           
            if not line or line.lower().replace('"', '') in ['judul,konten', 'title,content']:
                continue
            
            
            if line.count('"') > 4:
                print(f"DEBUG Line {line_num}: {line[:100]}...")

            
            parsed = parse_csv_line_robust(line)
            if parsed and len(parsed) == 2:
                judul, konten = parsed
                judul, konten = ensure_proper_quotes(judul, konten)
                
                writer.writerow([judul, konten])
                success_count += 1
                continue

            
            fail_count += 1
            failed_lines.append((line_num, line))

    print(f"-> ✅ Selesai. Berhasil: {success_count} baris, Gagal: {fail_count} baris.")
    print(f"   Hasil disimpan di: {output_path}")
    
    if failed_lines:
        print(f"\n❌ Baris yang gagal (total {len(failed_lines)} baris):")
        for i, (line_num, failed_line) in enumerate(failed_lines[:5]):
            print(f"   Baris {line_num}: {failed_line[:100]}...")

def validate_csv_format(file_path):
   
    try:
        df = pd.read_csv(file_path)
        print(f"\n✅ File berhasil dibaca oleh pandas!")
        print(f"   Shape: {df.shape}")
        print(f"   Kolom: {list(df.columns)}")
        print(f"\n📊 Sample data (3 baris pertama):")
        print("=" * 100)
        for i in range(min(3, len(df))):
            print(f"Baris {i+1}:")
            print(f"  JUDUL:  {df['judul'].iloc[i]}")
            print(f"  KONTEN: {df['konten'].iloc[i][:100]}...")
            print("-" * 100)
        
       
        print(f"\n🔍 Format file sebenarnya:")
        with open(file_path, 'r', encoding='utf-8') as f:
            for i, line in enumerate(f):
                if i < 4:  
                    print(f"Line {i}: {repr(line.strip())}")
                else:
                    break
        return True
    except Exception as e:
        print(f"❌ ERROR saat membaca file dengan pandas: {e}")
        return False

def create_test_file(input_path, output_test_path, num_lines=12):
    print(f"\n🔧 Membuat file test ({num_lines} baris pertama)...")
    
    success_count = 0
    
    with open(input_path, 'r', encoding='utf-8', errors='ignore') as infile, \
         open(output_test_path, 'w', encoding='utf-8', newline='') as outfile:
        
        writer = csv.writer(outfile, quoting=csv.QUOTE_ALL)
        writer.writerow(['judul', 'konten'])
        
        for line_num, line in enumerate(infile, 1):
            line = line.strip()
            
            if not line or line.lower().replace('"', '') in ['judul,konten', 'title,content']:
                continue
            
            if success_count >= num_lines:
                break
            
           
            print(f"\n--- Baris {line_num} (Original) ---")
            print(f"Raw: {repr(line)}")
            print(f"Quote count: {line.count(chr(34))}") 
            
            
            parsed = parse_csv_line_robust(line)
            if parsed and len(parsed) == 2:
                judul, konten = parsed
                judul, konten = ensure_proper_quotes(judul, konten)
                
                print(f"Judul (cleaned): {repr(judul)}")
                print(f"Konten (cleaned): {repr(konten[:50])}...")
                
                writer.writerow([judul, konten])
                success_count += 1
            else:
                print(f"❌ Gagal parsing baris {line_num}")

if __name__ == "__main__":
    input_filepath = os.path.join('datasets_cleaned', 'mojok.csv')
    output_test_path = os.path.join('cleaned', 'mojok_cleaned.csv')
    
    if not os.path.exists(input_filepath):
        print(f"❌ File input tidak ditemukan: {input_filepath}")
        print("   Pastikan file datasets_cleaned/mojok.csv ada")
    else:
        create_test_file(input_filepath, output_test_path, 9454)
        
        print("\n" + "="*80)
        print("VALIDASI FILE TEST:")
        print("="*80)
        validate_csv_format(output_test_path)
        
        print("\n" + "="*80)
        print("VALIDASI FILE LENGKAP:")
        print("="*80)
        validate_csv_format(output_filepath)

`Kompas-Dataset` - `Fix`

In [ ]:
def clean_excessive_quotes(text):
    if not text:
        return text
    
    text = text.strip()
    
    if text.count('"') >= 4:
        print(f"DEBUG: Banyak kutip ditemukan: {text[:100]}...")
    
  
    while text.startswith('"""'):
        text = text[3:]
    while text.startswith('""'):
        text = text[2:]
    if text.startswith('"'):
        text = text[1:]
    
 
    while text.endswith('"""'):
        text = text[:-3]
    while text.endswith('""'):
        text = text[:-2]
    if text.endswith('"'):
        text = text[:-1]
    
    return text.strip()

def remove_source_prefix(text):
   
    if not text:
        return text
    
    
    patterns = [
        r'^"\s*KOMPAS\.com\s*[–\-]\s*',
        r'^KOMPAS\.com\s*[–\-]\s*',
        r'^"\s*KOMPAS\.com\s*',
        r'^KOMPAS\.com\s*',
        r'^\(\s*KOMPAS\.com\s*\)\s*',
        r'^\[\s*KOMPAS\.com\s*\]\s*'
    ]
    
    cleaned_text = text.strip()
    
    for pattern in patterns:
        if re.match(pattern, cleaned_text, re.IGNORECASE):
            cleaned_text = re.sub(pattern, '', cleaned_text, flags=re.IGNORECASE)
            break
    
   
    cleaned_text = cleaned_text.replace('""', '"')
    
    return cleaned_text.strip()

def parse_csv_line_robust(line):
   
    line = line.strip()
    
    
    try:
        reader = csv.reader([line], quotechar='"', delimiter=',', quoting=csv.QUOTE_ALL, skipinitialspace=True)
        row = next(reader)
        if len(row) == 2:
            return [clean_excessive_quotes(row[0]), clean_excessive_quotes(row[1])]
    except Exception as e:
        pass
    
  
    match = re.match(r'^"([^"]*)"\s*,\s*"([^"]*)"$', line)
    if match:
        judul = clean_excessive_quotes(match.group(1))
        konten = clean_excessive_quotes(match.group(2))
        return [judul, konten]
    
    
    if line.count('"') >= 4:
       
        parts = re.split(r',\s*"', line, 1)
        if len(parts) == 2:
            judul = clean_excessive_quotes(parts[0])
            konten = '"' + parts[1] 
            konten = clean_excessive_quotes(konten)
            return [judul, konten]
    
   
    if ',' in line:
        first_comma = line.find(',')
        judul = line[:first_comma].strip()
        konten = line[first_comma+1:].strip()
        
        judul = clean_excessive_quotes(judul)
        konten = clean_excessive_quotes(konten)
        
        return [judul, konten]
    
    return None

def ensure_proper_quotes(judul, konten):
   
   
    judul = clean_excessive_quotes(judul)
    konten = clean_excessive_quotes(konten)
    
  
    konten = remove_source_prefix(konten)
    
    
    judul = judul.strip('"')
    konten = konten.strip('"')
    
    return judul, konten

def clean_and_split_csv(input_path, output_path):
    print(f"-> Memulai pembersihan untuk: {os.path.basename(input_path)}")

    success_count = 0
    fail_count = 0
    failed_lines = []

    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    with open(input_path, 'r', encoding='utf-8', errors='ignore') as infile, \
         open(output_path, 'w', encoding='utf-8', newline='') as outfile:

        writer = csv.writer(outfile, quoting=csv.QUOTE_ALL)  
        writer.writerow(['judul', 'konten'])

        for line_num, line in enumerate(infile, 1):
            line = line.strip()
            
           
            if not line or line.lower().replace('"', '') in ['judul,konten', 'title,content']:
                continue
            
            
            if line.count('"') > 4:
                print(f"DEBUG Line {line_num}: {line[:100]}...")

           
            parsed = parse_csv_line_robust(line)
            if parsed and len(parsed) == 2:
                judul, konten = parsed
                judul, konten = ensure_proper_quotes(judul, konten)
                
                writer.writerow([judul, konten])
                success_count += 1
                continue

           
            fail_count += 1
            failed_lines.append((line_num, line))

    print(f"-> ✅ Selesai. Berhasil: {success_count} baris, Gagal: {fail_count} baris.")
    print(f"   Hasil disimpan di: {output_path}")
    
    if failed_lines:
        print(f"\n❌ Baris yang gagal (total {len(failed_lines)} baris):")
        for i, (line_num, failed_line) in enumerate(failed_lines[:5]):
            print(f"   Baris {line_num}: {failed_line[:100]}...")

def validate_csv_format(file_path):

    try:
        df = pd.read_csv(file_path)
        print(f"\n✅ File berhasil dibaca oleh pandas!")
        print(f"   Shape: {df.shape}")
        print(f"   Kolom: {list(df.columns)}")
        print(f"\n📊 Sample data (3 baris pertama):")
        print("=" * 100)
        for i in range(min(3, len(df))):
            print(f"Baris {i+1}:")
            print(f"  JUDUL:  {df['judul'].iloc[i]}")
            print(f"  KONTEN: {df['konten'].iloc[i][:100]}...")
            print("-" * 100)
        
       
        print(f"\n🔍 Format file sebenarnya:")
        with open(file_path, 'r', encoding='utf-8') as f:
            for i, line in enumerate(f):
                if i < 4:  
                    print(f"Line {i}: {repr(line.strip())}")
                else:
                    break
        return True
    except Exception as e:
        print(f"❌ ERROR saat membaca file dengan pandas: {e}")
        return False

def create_test_file(input_path, output_test_path, num_lines=12):
    print(f"\n🔧 Membuat file test ({num_lines} baris pertama)...")
    
    success_count = 0
    
    with open(input_path, 'r', encoding='utf-8', errors='ignore') as infile, \
         open(output_test_path, 'w', encoding='utf-8', newline='') as outfile:
        
        writer = csv.writer(outfile, quoting=csv.QUOTE_ALL)
        writer.writerow(['judul', 'konten'])
        
        for line_num, line in enumerate(infile, 1):
            line = line.strip()
            
            if not line or line.lower().replace('"', '') in ['judul,konten', 'title,content']:
                continue
            
            if success_count >= num_lines:
                break
            
            
            print(f"\n--- Baris {line_num} (Original) ---")
            print(f"Raw: {repr(line)}")
            print(f"Quote count: {line.count(chr(34))}")  
            
            
            parsed = parse_csv_line_robust(line)
            if parsed and len(parsed) == 2:
                judul, konten = parsed
                judul, konten = ensure_proper_quotes(judul, konten)
                
                print(f"Judul (cleaned): {repr(judul)}")
                print(f"Konten (cleaned): {repr(konten[:50])}...")
                
                writer.writerow([judul, konten])
                success_count += 1
            else:
                print(f"❌ Gagal parsing baris {line_num}")


if __name__ == "__main__":
    input_filepath = os.path.join('datasets_cleaned', 'kompas.csv')
    output_test_path = os.path.join('cleaned', 'kompas_cleaned.csv')
    
    if not os.path.exists(input_filepath):
        print(f"❌ File input tidak ditemukan: {input_filepath}")
        print("   Pastikan file datasets_cleaned/kompas.csv ada")
    else:
        create_test_file(input_filepath, output_test_path, 9099)
        
        print("\n" + "="*80)
        print("VALIDASI FILE TEST:")
        print("="*80)
        validate_csv_format(output_test_path)
        
        # Validasi file lengkap
        print("\n" + "="*80)
        print("VALIDASI FILE LENGKAP:")
        print("="*80)
        validate_csv_format(output_filepath)

`Tempo Dataset` - `Fix`


In [ ]:
def clean_excessive_quotes(text):
    if not text:
        return text
    
    text = text.strip()
    
    if text.count('"') >= 4:
        print(f"DEBUG: Banyak kutip ditemukan: {text[:100]}...")
    
   
    while text.startswith('"""'):
        text = text[3:]
    while text.startswith('""'):
        text = text[2:]
    if text.startswith('"'):
        text = text[1:]
    
    while text.endswith('"""'):
        text = text[:-3]
    while text.endswith('""'):
        text = text[:-2]
    if text.endswith('"'):
        text = text[:-1]
    
    return text.strip()

def remove_source_prefix(text):
    if not text:
        return text
    
    patterns = [
        r'^"\s*KOMPAS\.com\s*[–\-]\s*',
        r'^KOMPAS\.com\s*[–\-]\s*',
        r'^"\s*KOMPAS\.com\s*',
        r'^KOMPAS\.com\s*',
        r'^\(\s*KOMPAS\.com\s*\)\s*',
        r'^\[\s*KOMPAS\.com\s*\]\s*'
    ]
    
    cleaned_text = text.strip()
    
    for pattern in patterns:
        if re.match(pattern, cleaned_text, re.IGNORECASE):
            cleaned_text = re.sub(pattern, '', cleaned_text, flags=re.IGNORECASE)
            break
    
   
    cleaned_text = cleaned_text.replace('""', '"')
    
    return cleaned_text.strip()

def parse_csv_line_robust(line):
   
    line = line.strip()
    
    
    try:
        reader = csv.reader([line], quotechar='"', delimiter=',', quoting=csv.QUOTE_ALL, skipinitialspace=True)
        row = next(reader)
        if len(row) == 2:
            return [clean_excessive_quotes(row[0]), clean_excessive_quotes(row[1])]
    except Exception as e:
        pass
    
   
    match = re.match(r'^"([^"]*)"\s*,\s*"([^"]*)"$', line)
    if match:
        judul = clean_excessive_quotes(match.group(1))
        konten = clean_excessive_quotes(match.group(2))
        return [judul, konten]
    
   
    if line.count('"') >= 4:
        
        parts = re.split(r',\s*"', line, 1)
        if len(parts) == 2:
            judul = clean_excessive_quotes(parts[0])
            konten = '"' + parts[1]  
            konten = clean_excessive_quotes(konten)
            return [judul, konten]
    
    
    if ',' in line:
        first_comma = line.find(',')
        judul = line[:first_comma].strip()
        konten = line[first_comma+1:].strip()
        
        judul = clean_excessive_quotes(judul)
        konten = clean_excessive_quotes(konten)
        
        return [judul, konten]
    
    return None

def ensure_proper_quotes(judul, konten):
   
   
    judul = clean_excessive_quotes(judul)
    konten = clean_excessive_quotes(konten)
    
   
    konten = remove_source_prefix(konten)
    
   
    judul = judul.strip('"')
    konten = konten.strip('"')
    
    return judul, konten

def clean_and_split_csv(input_path, output_path):
    print(f"-> Memulai pembersihan untuk: {os.path.basename(input_path)}")

    success_count = 0
    fail_count = 0
    failed_lines = []

    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    with open(input_path, 'r', encoding='utf-8', errors='ignore') as infile, \
         open(output_path, 'w', encoding='utf-8', newline='') as outfile:

        writer = csv.writer(outfile, quoting=csv.QUOTE_ALL) 
        writer.writerow(['judul', 'konten'])

        for line_num, line in enumerate(infile, 1):
            line = line.strip()
            
           
            if not line or line.lower().replace('"', '') in ['judul,konten', 'title,content']:
                continue
            
            
            if line.count('"') > 4:
                print(f"DEBUG Line {line_num}: {line[:100]}...")

           
            parsed = parse_csv_line_robust(line)
            if parsed and len(parsed) == 2:
                judul, konten = parsed
                judul, konten = ensure_proper_quotes(judul, konten)
                
                writer.writerow([judul, konten])
                success_count += 1
                continue

           
            fail_count += 1
            failed_lines.append((line_num, line))

    print(f"-> ✅ Selesai. Berhasil: {success_count} baris, Gagal: {fail_count} baris.")
    print(f"   Hasil disimpan di: {output_path}")
    
    if failed_lines:
        print(f"\n❌ Baris yang gagal (total {len(failed_lines)} baris):")
        for i, (line_num, failed_line) in enumerate(failed_lines[:5]):
            print(f"   Baris {line_num}: {failed_line[:100]}...")

def validate_csv_format(file_path):
    """Validasi format CSV hasil pembersihan"""
    try:
        df = pd.read_csv(file_path)
        print(f"\n✅ File berhasil dibaca oleh pandas!")
        print(f"   Shape: {df.shape}")
        print(f"   Kolom: {list(df.columns)}")
        print(f"\n📊 Sample data (3 baris pertama):")
        print("=" * 100)
        for i in range(min(3, len(df))):
            print(f"Baris {i+1}:")
            print(f"  JUDUL:  {df['judul'].iloc[i]}")
            print(f"  KONTEN: {df['konten'].iloc[i][:100]}...")
            print("-" * 100)
        
       
        print(f"\n🔍 Format file sebenarnya:")
        with open(file_path, 'r', encoding='utf-8') as f:
            for i, line in enumerate(f):
                if i < 4:  
                    print(f"Line {i}: {repr(line.strip())}")
                else:
                    break
        return True
    except Exception as e:
        print(f"❌ ERROR saat membaca file dengan pandas: {e}")
        return False

def create_test_file(input_path, output_test_path, num_lines=12):
    
    print(f"\n🔧 Membuat file test ({num_lines} baris pertama)...")
    
    success_count = 0
    
    with open(input_path, 'r', encoding='utf-8', errors='ignore') as infile, \
         open(output_test_path, 'w', encoding='utf-8', newline='') as outfile:
        
        writer = csv.writer(outfile, quoting=csv.QUOTE_ALL)
        writer.writerow(['judul', 'konten'])
        
        for line_num, line in enumerate(infile, 1):
            line = line.strip()
            
            if not line or line.lower().replace('"', '') in ['judul,konten', 'title,content']:
                continue
            
            if success_count >= num_lines:
                break
            
           
            print(f"\n--- Baris {line_num} (Original) ---")
            print(f"Raw: {repr(line)}")
            print(f"Quote count: {line.count(chr(34))}") 
            
            
            parsed = parse_csv_line_robust(line)
            if parsed and len(parsed) == 2:
                judul, konten = parsed
                judul, konten = ensure_proper_quotes(judul, konten)
                
                print(f"Judul (cleaned): {repr(judul)}")
                print(f"Konten (cleaned): {repr(konten[:50])}...")
                
                writer.writerow([judul, konten])
                success_count += 1
            else:
                print(f"❌ Gagal parsing baris {line_num}")

if __name__ == "__main__":
    input_filepath = os.path.join('datasets_cleaned', 'tempo.csv')
    output_test_path = os.path.join('cleaned', 'tempo_cleaned.csv')
    
    if not os.path.exists(input_filepath):
        print(f"❌ File input tidak ditemukan: {input_filepath}")
        print("   Pastikan file datasets_cleaned/tempo.csv ada")
    else:
        create_test_file(input_filepath, output_test_path, 10000)
        
        print("\n" + "="*80)
        print("VALIDASI FILE TEST:")
        print("="*80)
        validate_csv_format(output_test_path)
        
        print("\n" + "="*80)
        print("VALIDASI FILE LENGKAP:")
        print("="*80)
        validate_csv_format(output_filepath)

`Ugm Dataset` - `fix`

In [ ]:
def clean_excessive_quotes(text):
    if not text:
        return text
    
    text = text.strip()
    
    if text.count('"') >= 4:
        print(f"DEBUG: Banyak kutip ditemukan: {text[:100]}...")
    
    while text.startswith('"""'):
        text = text[3:]
    while text.startswith('""'):
        text = text[2:]
    if text.startswith('"'):
        text = text[1:]
    
    while text.endswith('"""'):
        text = text[:-3]
    while text.endswith('""'):
        text = text[:-2]
    if text.endswith('"'):
        text = text[:-1]
    
    return text.strip()

def remove_source_prefix(text):
    if not text:
        return text
    
    patterns = [
        r'^"\s*KOMPAS\.com\s*[–\-]\s*',
        r'^KOMPAS\.com\s*[–\-]\s*',
        r'^"\s*KOMPAS\.com\s*',
        r'^KOMPAS\.com\s*',
        r'^\(\s*KOMPAS\.com\s*\)\s*',
        r'^\[\s*KOMPAS\.com\s*\]\s*'
    ]
    
    cleaned_text = text.strip()
    
    for pattern in patterns:
        if re.match(pattern, cleaned_text, re.IGNORECASE):
            cleaned_text = re.sub(pattern, '', cleaned_text, flags=re.IGNORECASE)
            break
    
   
    cleaned_text = cleaned_text.replace('""', '"')
    
    return cleaned_text.strip()

def parse_csv_line_robust(line):
   
    line = line.strip()
    
   
    try:
        reader = csv.reader([line], quotechar='"', delimiter=',', quoting=csv.QUOTE_ALL, skipinitialspace=True)
        row = next(reader)
        if len(row) == 2:
            return [clean_excessive_quotes(row[0]), clean_excessive_quotes(row[1])]
    except Exception as e:
        pass
    
   
    match = re.match(r'^"([^"]*)"\s*,\s*"([^"]*)"$', line)
    if match:
        judul = clean_excessive_quotes(match.group(1))
        konten = clean_excessive_quotes(match.group(2))
        return [judul, konten]
    

    if line.count('"') >= 4:
       
        parts = re.split(r',\s*"', line, 1)
        if len(parts) == 2:
            judul = clean_excessive_quotes(parts[0])
            konten = '"' + parts[1]  
            konten = clean_excessive_quotes(konten)
            return [judul, konten]
    
    
    if ',' in line:
        first_comma = line.find(',')
        judul = line[:first_comma].strip()
        konten = line[first_comma+1:].strip()
        
        judul = clean_excessive_quotes(judul)
        konten = clean_excessive_quotes(konten)
        
        return [judul, konten]
    
    return None

def ensure_proper_quotes(judul, konten):
   
   
    judul = clean_excessive_quotes(judul)
    konten = clean_excessive_quotes(konten)
    
   
    konten = remove_source_prefix(konten)
    
   
    judul = judul.strip('"')
    konten = konten.strip('"')
    
    return judul, konten

def clean_and_split_csv(input_path, output_path):
    print(f"-> Memulai pembersihan untuk: {os.path.basename(input_path)}")

    success_count = 0
    fail_count = 0
    failed_lines = []

    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    with open(input_path, 'r', encoding='utf-8', errors='ignore') as infile, \
         open(output_path, 'w', encoding='utf-8', newline='') as outfile:

        writer = csv.writer(outfile, quoting=csv.QUOTE_ALL)  
        writer.writerow(['judul', 'konten'])

        for line_num, line in enumerate(infile, 1):
            line = line.strip()
            
            if not line or line.lower().replace('"', '') in ['judul,konten', 'title,content']:
                continue
            
           
            if line.count('"') > 4:
                print(f"DEBUG Line {line_num}: {line[:100]}...")

           
            parsed = parse_csv_line_robust(line)
            if parsed and len(parsed) == 2:
                judul, konten = parsed
                judul, konten = ensure_proper_quotes(judul, konten)
                
                writer.writerow([judul, konten])
                success_count += 1
                continue

            
            fail_count += 1
            failed_lines.append((line_num, line))

    print(f"-> ✅ Selesai. Berhasil: {success_count} baris, Gagal: {fail_count} baris.")
    print(f"   Hasil disimpan di: {output_path}")
    
    if failed_lines:
        print(f"\n❌ Baris yang gagal (total {len(failed_lines)} baris):")
        for i, (line_num, failed_line) in enumerate(failed_lines[:5]):
            print(f"   Baris {line_num}: {failed_line[:100]}...")

def validate_csv_format(file_path):
    """Validasi format CSV hasil pembersihan"""
    try:
        df = pd.read_csv(file_path)
        print(f"\n✅ File berhasil dibaca oleh pandas!")
        print(f"   Shape: {df.shape}")
        print(f"   Kolom: {list(df.columns)}")
        print(f"\n📊 Sample data (3 baris pertama):")
        print("=" * 100)
        for i in range(min(3, len(df))):
            print(f"Baris {i+1}:")
            print(f"  JUDUL:  {df['judul'].iloc[i]}")
            print(f"  KONTEN: {df['konten'].iloc[i][:100]}...")
            print("-" * 100)
        
        
        print(f"\n🔍 Format file sebenarnya:")
        with open(file_path, 'r', encoding='utf-8') as f:
            for i, line in enumerate(f):
                if i < 4:  
                    print(f"Line {i}: {repr(line.strip())}")
                else:
                    break
        return True
    except Exception as e:
        print(f"❌ ERROR saat membaca file dengan pandas: {e}")
        return False

def create_test_file(input_path, output_test_path, num_lines=12):
    print(f"\n🔧 Membuat file test ({num_lines} baris pertama)...")
    
    success_count = 0
    
    with open(input_path, 'r', encoding='utf-8', errors='ignore') as infile, \
         open(output_test_path, 'w', encoding='utf-8', newline='') as outfile:
        
        writer = csv.writer(outfile, quoting=csv.QUOTE_ALL)
        writer.writerow(['judul', 'konten'])
        
        for line_num, line in enumerate(infile, 1):
            line = line.strip()
            
            if not line or line.lower().replace('"', '') in ['judul,konten', 'title,content']:
                continue
            
            if success_count >= num_lines:
                break
            
        
            print(f"\n--- Baris {line_num} (Original) ---")
            print(f"Raw: {repr(line)}")
            print(f"Quote count: {line.count(chr(34))}") 
            
           
            parsed = parse_csv_line_robust(line)
            if parsed and len(parsed) == 2:
                judul, konten = parsed
                judul, konten = ensure_proper_quotes(judul, konten)
                
                print(f"Judul (cleaned): {repr(judul)}")
                print(f"Konten (cleaned): {repr(konten[:50])}...")
                
                writer.writerow([judul, konten])
                success_count += 1
            else:
                print(f"❌ Gagal parsing baris {line_num}")

if __name__ == "__main__":
    input_filepath = os.path.join('datasets_cleaned', 'etd_ugm.csv')
    output_test_path = os.path.join('cleaned', 'etd_ugm_cleaned.csv')
    
    if not os.path.exists(input_filepath):
        print(f"❌ File input tidak ditemukan: {input_filepath}")
        print("   Pastikan file datasets_cleaned/etd_ugm.csv ada")
    else:
        create_test_file(input_filepath, output_test_path, 50000)
        
        print("\n" + "="*80)
        print("VALIDASI FILE TEST:")
        print("="*80)
        validate_csv_format(output_test_path)
        
        print("\n" + "="*80)
        print("VALIDASI FILE LENGKAP:")
        print("="*80)
        validate_csv_format(output_filepath)

`Dataset etd_usk` - `Fix`

In [ ]:
def clean_excessive_quotes(text):
    
    if not text:
        return text
    
    text = text.strip()
    
    if text.count('"') >= 4:
        print(f"DEBUG: Banyak kutip ditemukan: {text[:100]}...")
    
   
    while text.startswith('"""'):
        text = text[3:]
    while text.startswith('""'):
        text = text[2:]
    if text.startswith('"'):
        text = text[1:]
    
    
    while text.endswith('"""'):
        text = text[:-3]
    while text.endswith('""'):
        text = text[:-2]
    if text.endswith('"'):
        text = text[:-1]
    
    return text.strip()

def remove_source_prefix(text):
   
    if not text:
        return text
    
   
    patterns = [
        r'^"\s*KOMPAS\.com\s*[–\-]\s*',
        r'^KOMPAS\.com\s*[–\-]\s*',
        r'^"\s*KOMPAS\.com\s*',
        r'^KOMPAS\.com\s*',
        r'^\(\s*KOMPAS\.com\s*\)\s*',
        r'^\[\s*KOMPAS\.com\s*\]\s*'
    ]
    
    cleaned_text = text.strip()
    
    for pattern in patterns:
        if re.match(pattern, cleaned_text, re.IGNORECASE):
            cleaned_text = re.sub(pattern, '', cleaned_text, flags=re.IGNORECASE)
            break
    
    
    cleaned_text = cleaned_text.replace('""', '"')
    
    return cleaned_text.strip()

def parse_csv_line_robust(line):
  
    line = line.strip()
    
   
    try:
        reader = csv.reader([line], quotechar='"', delimiter=',', quoting=csv.QUOTE_ALL, skipinitialspace=True)
        row = next(reader)
        if len(row) == 2:
            return [clean_excessive_quotes(row[0]), clean_excessive_quotes(row[1])]
    except Exception as e:
        pass
    
    match = re.match(r'^"([^"]*)"\s*,\s*"([^"]*)"$', line)
    if match:
        judul = clean_excessive_quotes(match.group(1))
        konten = clean_excessive_quotes(match.group(2))
        return [judul, konten]
    
   
    if line.count('"') >= 4:
       
        parts = re.split(r',\s*"', line, 1)
        if len(parts) == 2:
            judul = clean_excessive_quotes(parts[0])
            konten = '"' + parts[1]  
            konten = clean_excessive_quotes(konten)
            return [judul, konten]
    
    
    if ',' in line:
        first_comma = line.find(',')
        judul = line[:first_comma].strip()
        konten = line[first_comma+1:].strip()
        
        judul = clean_excessive_quotes(judul)
        konten = clean_excessive_quotes(konten)
        
        return [judul, konten]
    
    return None

def ensure_proper_quotes(judul, konten):
    
    judul = clean_excessive_quotes(judul)
    konten = clean_excessive_quotes(konten)
    
   
    konten = remove_source_prefix(konten)
    
   
    judul = judul.strip('"')
    konten = konten.strip('"')
    
    return judul, konten

def clean_and_split_csv(input_path, output_path):
    print(f"-> Memulai pembersihan untuk: {os.path.basename(input_path)}")

    success_count = 0
    fail_count = 0
    failed_lines = []

    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    with open(input_path, 'r', encoding='utf-8', errors='ignore') as infile, \
         open(output_path, 'w', encoding='utf-8', newline='') as outfile:

        writer = csv.writer(outfile, quoting=csv.QUOTE_ALL)  
        writer.writerow(['judul', 'konten'])

        for line_num, line in enumerate(infile, 1):
            line = line.strip()
            
          
            if not line or line.lower().replace('"', '') in ['judul,konten', 'title,content']:
                continue
            
           
            if line.count('"') > 4:
                print(f"DEBUG Line {line_num}: {line[:100]}...")

           
            parsed = parse_csv_line_robust(line)
            if parsed and len(parsed) == 2:
                judul, konten = parsed
                judul, konten = ensure_proper_quotes(judul, konten)
                
                writer.writerow([judul, konten])
                success_count += 1
                continue

            
            fail_count += 1
            failed_lines.append((line_num, line))

    print(f"-> ✅ Selesai. Berhasil: {success_count} baris, Gagal: {fail_count} baris.")
    print(f"   Hasil disimpan di: {output_path}")
    
    if failed_lines:
        print(f"\n❌ Baris yang gagal (total {len(failed_lines)} baris):")
        for i, (line_num, failed_line) in enumerate(failed_lines[:5]):
            print(f"   Baris {line_num}: {failed_line[:100]}...")

def validate_csv_format(file_path):
    try:
        df = pd.read_csv(file_path)
        print(f"\n✅ File berhasil dibaca oleh pandas!")
        print(f"   Shape: {df.shape}")
        print(f"   Kolom: {list(df.columns)}")
        print(f"\n📊 Sample data (3 baris pertama):")
        print("=" * 100)
        for i in range(min(3, len(df))):
            print(f"Baris {i+1}:")
            print(f"  JUDUL:  {df['judul'].iloc[i]}")
            print(f"  KONTEN: {df['konten'].iloc[i][:100]}...")
            print("-" * 100)
        
       
        print(f"\n🔍 Format file sebenarnya:")
        with open(file_path, 'r', encoding='utf-8') as f:
            for i, line in enumerate(f):
                if i < 4: 
                    print(f"Line {i}: {repr(line.strip())}")
                else:
                    break
        return True
    except Exception as e:
        print(f"❌ ERROR saat membaca file dengan pandas: {e}")
        return False

def create_test_file(input_path, output_test_path, num_lines=12):
    
    print(f"\n🔧 Membuat file test ({num_lines} baris pertama)...")
    
    success_count = 0
    
    with open(input_path, 'r', encoding='utf-8', errors='ignore') as infile, \
         open(output_test_path, 'w', encoding='utf-8', newline='') as outfile:
        
        writer = csv.writer(outfile, quoting=csv.QUOTE_ALL)
        writer.writerow(['judul', 'konten'])
        
        for line_num, line in enumerate(infile, 1):
            line = line.strip()
            
            if not line or line.lower().replace('"', '') in ['judul,konten', 'title,content']:
                continue
            
            if success_count >= num_lines:
                break
            
           
            print(f"\n--- Baris {line_num} (Original) ---")
            print(f"Raw: {repr(line)}")
            print(f"Quote count: {line.count(chr(34))}")  
            
           
            parsed = parse_csv_line_robust(line)
            if parsed and len(parsed) == 2:
                judul, konten = parsed
                judul, konten = ensure_proper_quotes(judul, konten)
                
                print(f"Judul (cleaned): {repr(judul)}")
                print(f"Konten (cleaned): {repr(konten[:50])}...")
                
                writer.writerow([judul, konten])
                success_count += 1
            else:
                print(f"❌ Gagal parsing baris {line_num}")


if __name__ == "__main__":
    input_filepath = os.path.join('datasets_cleaned', 'etd_usk.csv')
    output_test_path = os.path.join('cleaned', 'etd_usk_cleaned.csv')
    
    if not os.path.exists(input_filepath):
        print(f"❌ File input tidak ditemukan: {input_filepath}")
        print("   Pastikan file datasets_cleaned/tempo.csv ada")
    else:
        create_test_file(input_filepath, output_test_path, 10000)
        
        print("\n" + "="*80)
        print("VALIDASI FILE TEST:")
        print("="*80)
        validate_csv_format(output_test_path)

        print("\n" + "="*80)
        print("VALIDASI FILE LENGKAP:")
        print("="*80)
        validate_csv_format(output_filepath)

## Merge Dataset

In [ ]:
# Direktori tempat dataset asli Anda berada
input_dir = 'cleaned'

# Direktori untuk menyimpan hasil gabungan
output_dir = 'merge_datasets'

# Daftar nama file yang akan digabungkan
files_to_merge = [
    'etd_ugm_cleaned.csv', 
    'etd_usk_cleaned.csv', 
    'kompas_cleaned.csv', 
    'mojok_cleaned.csv', 
    'tempo_cleaned.csv'
]

# Nama file akhir setelah semua dataset digabungkan
output_filename = os.path.join(output_dir, '0_mergeDataset.csv')

# Ukuran chunk (kalau file besar banget)
chunk_size = 10000

print("Konfigurasi selesai:")
print(f"Input direktori: {input_dir}")
print(f"Output direktori: {output_dir}")
print(f"File output: {output_filename}")
print("="*60)

# Pastikan direktori output ada
os.makedirs(output_dir, exist_ok=True)

merged_df = pd.DataFrame()

# Proses penggabungan
for file_name in files_to_merge:
    file_path = os.path.join(input_dir, file_name)
    
    if not os.path.exists(file_path):
        print(f"❌ File tidak ditemukan: {file_name}")
        continue

    print(f"🔹 Membaca {file_name} ...")
    
    try:
        chunks = pd.read_csv(file_path, chunksize=chunk_size)
        for chunk in chunks:
            merged_df = pd.concat([merged_df, chunk], ignore_index=True)
    except Exception as e:
        print(f"⚠️ Gagal membaca {file_name}: {e}")
        continue

print("="*60)
print(f"✅ Total data gabungan: {merged_df.shape[0]} baris, {merged_df.shape[1]} kolom")

merged_df.to_csv(output_filename, index=False, quoting=1, encoding='utf-8')

print(f"💾 Hasil gabungan disimpan di: {output_filename}")


Konfigurasi selesai:
Input direktori: cleaned
Output direktori: merge_datasets
File output: merge_datasets\0_mergeDataset.csv
🔹 Membaca etd_ugm_cleaned.csv ...
🔹 Membaca etd_usk_cleaned.csv ...
🔹 Membaca kompas_cleaned.csv ...


🔹 Membaca mojok_cleaned.csv ...
🔹 Membaca tempo_cleaned.csv ...
✅ Total data gabungan: 46531 baris, 2 kolom
💾 Hasil gabungan disimpan di: merge_datasets\0_mergeDataset.csv
